# Week 3 Capstone Project

In [1]:
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

In [3]:
#  Import Libraries

from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


# Extracting data as text from URL

In [4]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data  = requests.get(url).text
soup = BeautifulSoup(html_data,"html5lib")
print('Done.')

Done.


In [5]:
# Finding Index of relevant table to be scraped

tables = soup.find_all('table')
for index,table in enumerate(tables):
    if ("M3A" in str(table)):
          table_index = index

print(table_index)

0


In [7]:
tor = pd.DataFrame(columns=["A"])

# Forming a data Frame using for and while loops 

for row in tables[table_index].tbody.find_all("tr"):
    i=0
    while i<9: 
#  As there are 9 colums in table

        col1 = row.find_all("td")
        if (col1 != []):

            a =col1[i].text
            i=i+1           
            tor = tor.append({"A":a}, ignore_index=True)        
tor.head(30)

,A
0,\nM1ANot assigned\n\n
1,\nM2ANot assigned\n\n
2,\nM3ANorth York(Parkwoods)\n\n
3,\nM4ANorth York(Victoria Village)\n\n
4,\nM5ADowntown Toronto(Regent Park / Harbourfro...
5,\nM6ANorth York(Lawrence Manor / Lawrence Heig...
6,\nM7AQueen's Park(Ontario Provincial Governmen...
7,\nM8ANot assigned\n\n
8,\nM9AEtobicoke(Islington Avenue)\n\n
9,\nM1BScarborough(Malvern / Rouge)\n\n


In [8]:
tor=tor[~tor.A.str.contains('Not assigned')]
tor.tail()

,A
160,\nM8XEtobicoke(The Kingsway / Montgomery Road ...
165,\nM4YDowntown Toronto(Church and Wellesley)\n\n
168,\nM7YEast TorontoBusiness reply mail Processin...
169,\nM8YEtobicoke(Old Mill South / King's Mill Pa...
178,\nM8ZEtobicoke(Mimico NW / The Queensway West ...


In [9]:
tor["A"]=tor["A"].str.replace("\n","")
tor.head()

,A
2,M3ANorth York(Parkwoods)
3,M4ANorth York(Victoria Village)
4,M5ADowntown Toronto(Regent Park / Harbourfront)
5,M6ANorth York(Lawrence Manor / Lawrence Heights)
6,M7AQueen's Park(Ontario Provincial Government)


In [10]:
# Splitting the dataframe A on the basis of postalcode 

tor['PostalCode']=tor['A'].str.slice(stop=3)
tor['bn']=tor['A'].str.slice(start=3)

tor.drop(columns="A",inplace=True)
tor.head()

,PostalCode,bn
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)
5,M6A,North York(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park(Ontario Provincial Government)


In [11]:
# Splitting the borrough and neighbourhood

to=tor["bn"].str.split("(",n=1,expand=True)
to.head()

,0,1
2,North York,Parkwoods)
3,North York,Victoria Village)
4,Downtown Toronto,Regent Park / Harbourfront)
5,North York,Lawrence Manor / Lawrence Heights)
6,Queen's Park,Ontario Provincial Government)


In [12]:
#  Creating new dataframe with desired columns 

data = pd.DataFrame(columns=["PostalCode", "Borough","Neighbourhood"])

data["PostalCode"]=tor["PostalCode"]
data["Borough"]=to[0]
data["Neighbourhood"]=to[1]

data["Neighbourhood"]=data["Neighbourhood"].str.replace(')', '')

data

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don MillsNorth
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
data.shape

(103, 3)

## Required data set has been scrapped from wikipedia webpage with desired columns as shown above